In [49]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

!pip install https://github.com/pytorch/text/archive/master.zip

     \ 921kB 19.4MB/s
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-6sde3hjw/wheels/5a/86/3d/30ae7dfdfeb1748bb11b3da173fb9634141fbb39e9e9847317
Successfully built torchtext


In [50]:
!pip install https://github.com/pytorch/text/archive/master.zip

     \ 880kB 20.0MB/s
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-ulx3tocs/wheels/5a/86/3d/30ae7dfdfeb1748bb11b3da173fb9634141fbb39e9e9847317
Successfully built torchtext


In [0]:
import torch
import torch.nn as nn 
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.datasets as dsets 
import torchvision.transforms as transforms
import torchtext 
from torchtext import datasets 
from torchtext import data
from gensim.models import KeyedVectors
import gensim
import nltk
import random

In [0]:
SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [53]:
print(torch.cuda.is_available())

True


In [54]:
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.300d")
LABEL.build_vocab(train_data)
dir(TEXT)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'batch_first',
 'build_vocab',
 'dtype',
 'dtypes',
 'eos_token',
 'fix_length',
 'ignore',
 'include_lengths',
 'init_token',
 'is_target',
 'lower',
 'numericalize',
 'pad',
 'pad_first',
 'pad_token',
 'postprocessing',
 'preprocess',
 'preprocessing',
 'process',
 'sequential',
 'stop_words',
 'tokenize',
 'tokenizer_args',
 'truncate_first',
 'unk_token',
 'use_vocab',
 'vocab',
 'vocab_cls']

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=BATCH_SIZE, device=device)


In [56]:
for batch in train_iterator:
    print(dir(batch))
    print(batch.fields)
    break

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_get_field_values', 'batch_size', 'dataset', 'fields', 'fromvars', 'input_fields', 'label', 'target_fields', 'text']
dict_keys(['text', 'label'])


In [0]:
class lstmclassifier(nn.Module):
    def __init__(self,input_dim, vocab_size, hidden_dim, output_dim, embedding_dim):
        super(lstmclassifier, self).__init__() 
        self.hidden_dim = hidden_dim 
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        x = x.permute(1,0)
        x = self.embedding(x)
        #print(x.shape)  #(batch_size, sequence_length, input_length)
        x = x.permute(1,0,2) #(seq_len, batch_size, input_dim)
        if torch.cuda.is_available():
          h0 = Variable(torch.zeros(1, x.size(1), self.hidden_dim).cuda())
        else:
          h0 = Variable(torch.zeros(1, x.size(1), self.hidden_dim))
        if torch.cuda.is_available():
          c0 = Variable(torch.zeros(1,x.size(1), self.hidden_dim).cuda())
        else:
          c0 = Variable(torch.zeros(1,x.size(1), self.hidden_dim)) 
        out,(hn,cn) = self.lstm(x,(h0,c0))
        #print(out.shape)
        #final_output = self.linear(out[-1])
        final_output = F.relu(self.linear(out[-1]))
        #print(out[-1].shape)
        return final_output
        #final_output = self.linear(h0[-1])
        

In [64]:
input_dim = len(TEXT.vocab)
hidden_dim = 128 
output_dim = 1 
embedding_dim = 300 
vocab_size = input_dim

model = lstmclassifier(input_dim , vocab_size, hidden_dim, output_dim, embedding_dim)
if torch.cuda.is_available():
  model.cuda()
optimizer = torch.optim.Adadelta(model.parameters())

criterion = nn.BCEWithLogitsLoss()

pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1957,  0.5752, -0.1345,  ..., -0.2973,  0.1073, -0.5232]],
       device='cuda:0')

In [0]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [65]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.693 | Train Acc: 49.64% | Val. Loss: 0.693 | Val. Acc: 50.79% |
| Epoch: 02 | Train Loss: 0.693 | Train Acc: 49.67% | Val. Loss: 0.693 | Val. Acc: 50.79% |
| Epoch: 03 | Train Loss: 0.693 | Train Acc: 49.67% | Val. Loss: 0.693 | Val. Acc: 50.79% |
| Epoch: 04 | Train Loss: 0.693 | Train Acc: 49.67% | Val. Loss: 0.693 | Val. Acc: 50.79% |
| Epoch: 05 | Train Loss: 0.693 | Train Acc: 49.67% | Val. Loss: 0.693 | Val. Acc: 50.79% |
